In [1]:
from selenium import webdriver
import time
import pickle
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import numpy as np
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images":2})
chrome_options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 2})
chrome_options.add_experimental_option("prefs", {"profile.default_content_settings.popups": 1})
#chrome_options.add_extension('C:/Users/Schmidt/Anaconda3/selenium/webdriver/AdBlock_v3.49.1.crx')

path_to_chromedriver = "C:/Users/Schmidt\Anaconda3\selenium\webdriver\chromedriver.exe"
#path_to_chromedriver = "C:/Users/Peter\Anaconda3\selenium\webdriver\chromedriver.exe"

# Scrape Match Data

In [3]:
def get_match_vars():

    match_vars = {}
    
    match_vars['league'] = league
    match_vars['season'] = season
    
    temp = browser.find_element_by_xpath(
        '//*[@id="match-header"]/table/tbody/tr[2]/td[2]/div[3]/dl/dd[2]').text
    match_vars['weekday'] = temp.split(',')[0]
    day = temp.split(' ')[1].split('-')[0]
    month = temp.split(' ')[1].split('-')[1]
    year = '20' + temp.split(' ')[1].split('-')[2]
    match_vars['date'] = year + '-' + str(datetime.strptime(month, '%b').month) + '-' + day
    
    match_vars['start_time'] = browser.find_element_by_xpath(
        '//*[@id="match-header"]/table/tbody/tr[2]/td[2]/div[3]/dl/dd[1]').text

    return match_vars

In [4]:
def get_team_vars():

    team_vars = {}
    team_vars['home'] = {}
    team_vars['away'] = {}

    team_vars['home']['team_name'] = browser.find_element_by_xpath(
        '//*[@id="match-header"]/table/tbody/tr[1]/td[1]/a').get_attribute("text")
    team_vars['away']['team_name'] = browser.find_element_by_xpath(
        '//*[@id="match-header"]/table/tbody/tr[1]/td[3]/a').get_attribute("text")

    score_final = browser.find_element_by_xpath(
        '//*[@id="match-header"]/table/tbody/tr[1]/td[2]').text
    team_vars['home']['goals'] = int(score_final.split(':')[0].split(' ')[0])
    team_vars['away']['goals'] = int(score_final.split(':')[1].split(' ')[1])
    team_vars['home']['opp_goals'] = int(score_final.split(':')[1].split(' ')[1])
    team_vars['away']['opp_goals'] = int(score_final.split(':')[0].split(' ')[0])
    
    if team_vars['home']['goals'] > team_vars['away']['goals']:
        results = ['W', 'L']
    if team_vars['home']['goals'] < team_vars['away']['goals']:
        results = ['L', 'W']
    if team_vars['home']['goals'] == team_vars['away']['goals']:
        results = ['D', 'D']
    team_vars['home']['result'] = results[0]
    team_vars['away']['result'] = results[1]
    
    team_vars['home']['shots'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[1]/span[1]/span').text)
    team_vars['away']['shots'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[1]/span[3]/span').text)
    
    team_vars['home']['shots_ot'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[2]/span[1]/span').text)
    team_vars['away']['shots_ot'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[2]/span[3]/span').text)
    
    team_vars['home']['pass_succ'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[3]/span[1]/span').text.replace('%',''))
    team_vars['away']['pass_succ'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[3]/span[3]/span').text.replace('%',''))
    
    team_vars['home']['air_duel_rate'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[4]/span[1]/span').text.replace('%',''))
    team_vars['away']['air_duel_rate'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[4]/span[3]/span').text.replace('%',''))

    team_vars['home']['dribbles_won'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[5]/span[1]/span').text)
    team_vars['away']['dribbles_won'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[5]/span[3]/span').text)
    
    team_vars['home']['tackles'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[6]/span[1]/span').text)
    team_vars['away']['tackles'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[1]/div[6]/span[3]/span').text)
    
    team_vars['home']['possession'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[2]/div[2]/span/span[2]/span').text.replace('%',''))
    team_vars['away']['possession'] = int(browser.find_element_by_xpath(
        '//*[@id="match-report-team-statistics"]/div[2]/div[2]/span/span[3]/span').text.replace('%',''))
    
    return team_vars

In [5]:
def click_section(side, section):
    path = "//*[@id='live-player-" + side + "-options']/li/a[text() =" + "'" + section + "'" + "]"
    browser.find_element_by_xpath(path).location_once_scrolled_into_view
    browser.find_element_by_xpath(path).click()

In [6]:
def get_events(idx, side, event_dict):
    
    player_event_dict = {}
    
    parent = browser.find_element_by_id('live-player-' + side + '-summary')
    path = './/*[@id="player-table-statistics-body"]/tr[' + str(idx + 1) + ']/td[11]/span/span'
    events = parent.find_elements_by_xpath(path)

    for counter in range(len(events)):
        event_type = events[counter].get_attribute('data-type')
        
        if event_type == '16' and 'goalown' in events[counter].get_attribute('outerHTML'):
            event_type = '99'
        if event_type == '17' and events[counter].get_attribute('data-card-type') == '33':
            event_type = '98'        
        
        if event_type in event_dict.keys():
        
            if event_dict[event_type] in player_event_dict.keys():
                player_event_dict[event_dict[event_type]] +=1
            else:
                player_event_dict[event_dict[event_type]] = 1
        
    return player_event_dict

In [7]:
def get_min_played(p_rating, p_name_temp, idx):
    if p_rating == '-':
        p_min_played = 0
    else:
        if len(p_name_temp.split('(')) == 1:
            p_min_played = 90
            
        if len(p_name_temp.split('(')) == 2:
            if idx <= 10:
                p_min_played = int(p_name_temp.split(' (')[1].replace('′','').replace(')',''))
            else:
                p_min_played = 90 - int(p_name_temp.split(' (')[1].replace('′','').replace(')',''))
                
        if len(p_name_temp.split('(')) == 3:
            sub_out = p_name_temp.split('(')[1].replace('′','').replace(')','')
            sub_in = p_name_temp.split('(')[2].replace('′','').replace(')','')
            p_min_played = int(sub_out) - int(sub_in)
    
    if p_min_played < 0:
        p_min_played = 1
    
    return p_min_played

In [8]:
def get_player_vars(side):
    sections = {'Summary': ['p_shots', 'p_shots_ot', 'p_key_passes', 'p_pass_acc', 'p_aerials_won', 'p_touches'],
                'Offensive': ['p_shots', 'p_shots_ot', 'p_key_passes', 'p_dribbles', 'p_fouled', 'p_offsides',
                              'p_disp', 'p_bad_controls'], 
                'Defensive': ['p_total_tackles', 'p_interceptions', 'p_clears', 'p_blocks', 'p_fouls'], 
                'Passing':   ['p_key_passes', 'p_passes', 'p_pass_acc', 'p_crosses', 'p_acc_crosses', 'p_long_balls', 
                              'p_acc_long_balls', 'p_through_balls', 'p_acc_through_balls']}

    player_vars = pd.DataFrame()

    click_section(side, 'Summary')

    temp_table = browser.find_element_by_id('live-player-' + side + '-summary').text
    nr_players = int(((len(str.splitlines(temp_table))) - 3) / 3)

    for section in sections.keys():

        click_section(side, section)

        time.sleep(1)

        rawstring = browser.find_element_by_id('live-player-' + side + '-' + str.lower(section)).text
        table = str.splitlines(rawstring)

        for idx in range(nr_players):
            p_name_temp = table[2 + 3*idx]
            p_name = p_name_temp.split('(')[0]
            if p_name[-1] == ' ':
                p_name = p_name[:-1]
                
            p_rating_temp = table[3 + 3*idx].split(' ')[-1]
            try:
                p_rating = float(p_rating_temp)
            except ValueError:
                p_rating = np.nan
                                 
            for p_var_idx, p_var in enumerate(sections[section]):
                p_stats = table[3 + 3*idx]
                player_vars.loc[p_name, p_var] = p_stats.split(' ')[2 + p_var_idx]
            
            p_min_played = get_min_played(p_rating, p_name_temp, idx)

            player_vars.loc[p_name, 'p_rating'] = p_rating
            player_vars.loc[p_name, 'p_age'] = int(table[3 + 3*idx].split(',')[0])
            player_vars.loc[p_name, 'p_position'] = table[3 + 3*idx].split(', ')[1].split(' ')[0]
            player_vars.loc[p_name, 'p_min_played'] = p_min_played
            player_vars.loc[p_name, 'p_name'] = p_name
            player_vars.loc[p_name, 'matchid'] = match_id
            
            if side == 'home':
                player_vars.loc[p_name, 'home'] = 1
            if side == 'away':
                player_vars.loc[p_name, 'home'] = 0

            
            event_dict = {'1': 'p_assists', '3': 'p_assists', '7': 'p_tackles_last_man', '8': 'p_assists', 
                          '10': 'p_clears_off_line', '12': 'p_assists', 
                          '13': 'p_assists', '14': 'p_post_crossbar', '15': 'p_penalty_missed', '16': 'p_goals', 
                          '17': 'p_yellow_cards', '51': 'p_errors', '58': 'p_penalty_saved', '61': 'p_assists', 
                          '98': 'p_red_card', '99': 'p_own_goals'}

            for col in set(list(event_dict.values())):
                player_vars.loc[p_name, col] = 0

            player_events = get_events(idx, side, event_dict)

            for event_var in player_events.keys():
                player_vars.loc[p_name, event_var] = player_events[event_var]
    
    player_vars.reset_index(drop = True, inplace = True)
    return player_vars

In [9]:
def accept_cookies():
    cookies_button = browser.find_elements_by_xpath('//*[@id="qcCmpButtons"]/button')
    if len(cookies_button) > 0:
        cookies_button[0].click()
        time.sleep(1)

In [10]:
def scrape_game(url):
    browser.get(url)
    time.sleep(2)

    accept_cookies()

    match_vars = get_match_vars()
    team_vars = get_team_vars()
    
    match_df = pd.DataFrame()
    
    for side in ['home', 'away']:
        side_df = get_player_vars(side)

        for match_var in match_vars.keys():
            side_df[match_var] = match_vars[match_var]
        for team_var in team_vars[side].keys():
            side_df[team_var] = team_vars[side][team_var]
        
        match_df = match_df.append(side_df)
                   
    return match_df    

In [11]:
def print_success(match_idx, matchid_container):
    print(str(match_idx + 1) + ' out of ' + str(len(matchid_container)) + ' matches in ' + str(league) + ' ' + str(season) + ' complete.')

In [12]:
def print_failure(match_idx, matchid_container):
    print('match dl failed for match with id ' + str(matchid_container[match_idx]))

In [58]:
browser = webdriver.Chrome(executable_path=path_to_chromedriver, chrome_options=chrome_options)

league = 'prem_league'
season = 'test'
match_id = '1285057'

url = "https://www.whoscored.com/Matches/" + match_id + "/Livestatistics/"

test_df = scrape_game(url)

In [14]:
missing_games = {}

for league in ['bundesliga', 'serie_a', 'prem_league', 'laliga', 'ligue1']:
    missing_games[league] = {}
    for season in ['09_10', '10_11', '11_12', '12_13', '13_14', '14_15', '15_16', '16_17', '17_18', '18_19']:
        all_games = match_ids[league][season]
        scraped_games = list(df.loc[(df.league == league) & (df.season == season), 'matchid'].unique())
        missing_games[league][season] = list(set(all_games) - set(scraped_games))
        print(league + ' ' + season + ': ' + str(len(missing_games[league][season])))

bundesliga 09_10: 0
bundesliga 10_11: 0
bundesliga 11_12: 0
bundesliga 12_13: 0
bundesliga 13_14: 0
bundesliga 14_15: 0
bundesliga 15_16: 0
bundesliga 16_17: 0
bundesliga 17_18: 0
bundesliga 18_19: 0
serie_a 09_10: 0
serie_a 10_11: 0
serie_a 11_12: 1
serie_a 12_13: 1
serie_a 13_14: 0
serie_a 14_15: 0
serie_a 15_16: 0
serie_a 16_17: 1
serie_a 17_18: 1
serie_a 18_19: 0
prem_league 09_10: 0
prem_league 10_11: 0
prem_league 11_12: 1
prem_league 12_13: 0
prem_league 13_14: 0
prem_league 14_15: 0
prem_league 15_16: 0
prem_league 16_17: 0
prem_league 17_18: 0
prem_league 18_19: 0
laliga 09_10: 0
laliga 10_11: 0
laliga 11_12: 0
laliga 12_13: 0
laliga 13_14: 0
laliga 14_15: 0
laliga 15_16: 0
laliga 16_17: 0
laliga 17_18: 0
laliga 18_19: 0
ligue1 09_10: 0
ligue1 10_11: 0
ligue1 11_12: 0
ligue1 12_13: 0
ligue1 13_14: 0
ligue1 14_15: 0
ligue1 15_16: 0
ligue1 16_17: 1
ligue1 17_18: 0
ligue1 18_19: 0


In [13]:
match_ids = pickle.load(open('match_ids.pckl', 'rb'))

df = pickle.load(open('player_level.pckl', 'rb'))

In [14]:
df.league.unique()

array(['bundesliga', 'prem_league', 'serie_a', 'laliga', 'ligue1',
       'eredivisie', 'superlig'], dtype=object)

In [17]:
match_ids.keys()

dict_keys(['laliga', 'bundesliga', 'prem_league', 'serie_a', 'ligue1', 'eredivisie', 'liga_nos', 'superlig'])

In [15]:
match_ids['liga_nos'].keys()

dict_keys(['16_17', '17_18', '18_19'])

In [16]:
browser = webdriver.Chrome(executable_path=path_to_chromedriver, chrome_options=chrome_options)

for league in ['liga_nos']:
    for season in ['16_17', '17_18', '18_19']:

        if not df.empty:
            already_scraped = list(df.matchid.unique())
        else:
            already_scraped = []    

        matchids_to_scrape = list(set(match_ids[league][season]) - set(already_scraped))

        for match_id in matchids_to_scrape:
            url = "https://www.whoscored.com/Matches/" + match_id + "/Livestatistics/"

            fails = 0
            success = False

            try:
                match_df = scrape_game(url)
                success = True

            except KeyboardInterrupt:
                raise

            except:
                while fails < 3 and not success:
                    try:
                        time.sleep(2)
                        match_df = scrape_game(url)
                        success = True
                    except KeyboardInterrupt:
                        raise
                    except:
                        fails += 1
                        print('first loop, fail nr ' + str(fails))

                while fails < 5 and not success:
                    try:
                        browser = webdriver.Chrome(executable_path=path_to_chromedriver, chrome_options=chrome_options)
                        time.sleep(10)
                        match_df = scrape_game(url)
                        success = True

                    except KeyboardInterrupt:
                        raise
                    except:
                        fails += 1
                        print('second loop, fail nr ' + str(fails))

            if success:
                df = df.append(match_df)
                pickle.dump(df, open("player_level.pckl", "wb" ))
                print_success(matchids_to_scrape.index(match_id), matchids_to_scrape)     
            else:
                print_failure(matchids_to_scrape.index(match_id), matchids_to_scrape)

C:\Users\Schmidt\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1 out of 306 matches in liga_nos 16_17 complete.
2 out of 306 matches in liga_nos 16_17 complete.
3 out of 306 matches in liga_nos 16_17 complete.
4 out of 306 matches in liga_nos 16_17 complete.
5 out of 306 matches in liga_nos 16_17 complete.
6 out of 306 matches in liga_nos 16_17 complete.
7 out of 306 matches in liga_nos 16_17 complete.
8 out of 306 matches in liga_nos 16_17 complete.
9 out of 306 matches in liga_nos 16_17 complete.
10 out of 306 matches in liga_nos 16_17 complete.
11 out of 306 matches in liga_nos 16_17 complete.
12 out of 306 matches in liga_nos 16_17 complete.
13 out of 306 matches in liga_nos 16_17 complete.
14 out of 306 matches in liga_nos 16_17 complete.
15 out of 306 matches in liga_nos 16_17 complete.
16 out of 306 matches in liga_nos 16_17 complete.
17 out of 306 matches in liga_nos 16_17 complete.
18 out of 306 matches in liga_nos 16_17 complete.
19 out of 306 matches in liga_nos 16_17 complete.
20 out of 306 matches in liga_nos 16_17 complete.
21 out of

164 out of 306 matches in liga_nos 16_17 complete.
165 out of 306 matches in liga_nos 16_17 complete.
166 out of 306 matches in liga_nos 16_17 complete.
167 out of 306 matches in liga_nos 16_17 complete.
168 out of 306 matches in liga_nos 16_17 complete.
169 out of 306 matches in liga_nos 16_17 complete.
170 out of 306 matches in liga_nos 16_17 complete.
171 out of 306 matches in liga_nos 16_17 complete.
172 out of 306 matches in liga_nos 16_17 complete.
173 out of 306 matches in liga_nos 16_17 complete.
174 out of 306 matches in liga_nos 16_17 complete.
first loop, fail nr 1
175 out of 306 matches in liga_nos 16_17 complete.
176 out of 306 matches in liga_nos 16_17 complete.
177 out of 306 matches in liga_nos 16_17 complete.
178 out of 306 matches in liga_nos 16_17 complete.
179 out of 306 matches in liga_nos 16_17 complete.
180 out of 306 matches in liga_nos 16_17 complete.
181 out of 306 matches in liga_nos 16_17 complete.
182 out of 306 matches in liga_nos 16_17 complete.
183 out o

17 out of 306 matches in liga_nos 17_18 complete.
18 out of 306 matches in liga_nos 17_18 complete.
19 out of 306 matches in liga_nos 17_18 complete.
20 out of 306 matches in liga_nos 17_18 complete.
21 out of 306 matches in liga_nos 17_18 complete.
22 out of 306 matches in liga_nos 17_18 complete.
23 out of 306 matches in liga_nos 17_18 complete.
24 out of 306 matches in liga_nos 17_18 complete.
25 out of 306 matches in liga_nos 17_18 complete.
26 out of 306 matches in liga_nos 17_18 complete.
27 out of 306 matches in liga_nos 17_18 complete.
28 out of 306 matches in liga_nos 17_18 complete.
29 out of 306 matches in liga_nos 17_18 complete.
30 out of 306 matches in liga_nos 17_18 complete.
31 out of 306 matches in liga_nos 17_18 complete.
32 out of 306 matches in liga_nos 17_18 complete.
33 out of 306 matches in liga_nos 17_18 complete.
34 out of 306 matches in liga_nos 17_18 complete.
35 out of 306 matches in liga_nos 17_18 complete.
36 out of 306 matches in liga_nos 17_18 complete.


178 out of 306 matches in liga_nos 17_18 complete.
179 out of 306 matches in liga_nos 17_18 complete.
180 out of 306 matches in liga_nos 17_18 complete.
181 out of 306 matches in liga_nos 17_18 complete.
182 out of 306 matches in liga_nos 17_18 complete.
183 out of 306 matches in liga_nos 17_18 complete.
184 out of 306 matches in liga_nos 17_18 complete.
185 out of 306 matches in liga_nos 17_18 complete.
186 out of 306 matches in liga_nos 17_18 complete.
187 out of 306 matches in liga_nos 17_18 complete.
188 out of 306 matches in liga_nos 17_18 complete.
189 out of 306 matches in liga_nos 17_18 complete.
190 out of 306 matches in liga_nos 17_18 complete.
191 out of 306 matches in liga_nos 17_18 complete.
192 out of 306 matches in liga_nos 17_18 complete.
193 out of 306 matches in liga_nos 17_18 complete.
194 out of 306 matches in liga_nos 17_18 complete.
195 out of 306 matches in liga_nos 17_18 complete.
196 out of 306 matches in liga_nos 17_18 complete.
197 out of 306 matches in liga_

33 out of 306 matches in liga_nos 18_19 complete.
34 out of 306 matches in liga_nos 18_19 complete.
35 out of 306 matches in liga_nos 18_19 complete.
36 out of 306 matches in liga_nos 18_19 complete.
37 out of 306 matches in liga_nos 18_19 complete.
38 out of 306 matches in liga_nos 18_19 complete.
39 out of 306 matches in liga_nos 18_19 complete.
40 out of 306 matches in liga_nos 18_19 complete.
41 out of 306 matches in liga_nos 18_19 complete.
42 out of 306 matches in liga_nos 18_19 complete.
43 out of 306 matches in liga_nos 18_19 complete.
44 out of 306 matches in liga_nos 18_19 complete.
first loop, fail nr 1
first loop, fail nr 2
first loop, fail nr 3
45 out of 306 matches in liga_nos 18_19 complete.
46 out of 306 matches in liga_nos 18_19 complete.
47 out of 306 matches in liga_nos 18_19 complete.
48 out of 306 matches in liga_nos 18_19 complete.
first loop, fail nr 1
49 out of 306 matches in liga_nos 18_19 complete.
50 out of 306 matches in liga_nos 18_19 complete.
51 out of 30

192 out of 306 matches in liga_nos 18_19 complete.
193 out of 306 matches in liga_nos 18_19 complete.
first loop, fail nr 1
194 out of 306 matches in liga_nos 18_19 complete.
195 out of 306 matches in liga_nos 18_19 complete.
196 out of 306 matches in liga_nos 18_19 complete.
197 out of 306 matches in liga_nos 18_19 complete.
198 out of 306 matches in liga_nos 18_19 complete.
199 out of 306 matches in liga_nos 18_19 complete.
200 out of 306 matches in liga_nos 18_19 complete.
201 out of 306 matches in liga_nos 18_19 complete.
202 out of 306 matches in liga_nos 18_19 complete.
203 out of 306 matches in liga_nos 18_19 complete.
204 out of 306 matches in liga_nos 18_19 complete.
205 out of 306 matches in liga_nos 18_19 complete.
206 out of 306 matches in liga_nos 18_19 complete.
207 out of 306 matches in liga_nos 18_19 complete.
208 out of 306 matches in liga_nos 18_19 complete.
209 out of 306 matches in liga_nos 18_19 complete.
210 out of 306 matches in liga_nos 18_19 complete.
211 out o